In [ ]:
# SKRYPT EKSPERYMENTALNY
# Uruchamia analizę algorytmu Tabu Search (TS) dla różnych
# konfiguracji parametrów. Dla każdej kombinacji wykonywane jest
# kilka powtórzeń, zapisywane są statystyki jakości oraz czas
# działania. Wyniki trafiają do pliku CSV.

%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
import itertools
from multiprocessing import Pool, cpu_count

from src.utils.tsp_loader import load_tsp_matrix
from algorithms.tabu_move import solve_tsp
# from src.algorithms.tabu_full_path import solve_tsp
from src.utils.result_saver import save_experiment_results
from src.utils.run_single_repeat import run_single_repeat


# USTAWIENIA
TSP_FILES = ["Dane_TSP_48.xlsx", "Dane_TSP_76.xlsx", "Dane_TSP_127.xlsx"]

PARAM_GRID = {
    "max_iter": [5_000, 10_000, 25_000, 100_000],
    "stop_no_improve": [500, 1_000, 2_000, 5_000],
    "tabu_tenure": [5, 10, 15, 25],
    "n_neighbors": [10, 30, 60, 100],
    "neighborhood_type": ["swap", "insert", "two_opt"],
}

REPEATS = 5
results = []


# ROZGRZANIE (KOMPILACJA JIT JEŚLI WYSTĘPUJE)
print("Rozgrzewanie (kompilacja JIT jeśli dotyczy)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {"max_iter": 5, "tabu_tenure": 3, "n_neighbors": 5})
print("Kompilacja zakończona.\n")


# GŁÓWNA PĘTLA
# Obliczamy łączną liczbę kombinacji
all_combos = list(itertools.product(
    PARAM_GRID["max_iter"],
    PARAM_GRID["stop_no_improve"],
    PARAM_GRID["tabu_tenure"],
    PARAM_GRID["n_neighbors"],
    PARAM_GRID["neighborhood_type"],
))
total = len(all_combos) * len(TSP_FILES)
counter = 0
start_total = time.perf_counter()

for tsp_file in TSP_FILES:
    print(f"\nInstancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for max_iter in PARAM_GRID["max_iter"]:
        for stop_no_improve in PARAM_GRID["stop_no_improve"]:
            for tabu_tenure in PARAM_GRID["tabu_tenure"]:
                for n_neighbors in PARAM_GRID["n_neighbors"]:
                    for neighborhood_type in PARAM_GRID["neighborhood_type"]:
                        counter += 1

                        print(
                            f"[{counter}/{total}] "
                            f"max_iter={max_iter}, "
                            f"stop_no_improve={stop_no_improve}, "
                            f"tabu_tenure={tabu_tenure}, "
                            f"n_neighbors={n_neighbors}"
                            f"neighborhood_type={neighborhood_type}"
                        )

                        params = {
                            "max_iter": max_iter,
                            "stop_no_improve": stop_no_improve,
                            "tabu_tenure": tabu_tenure,
                            "n_neighbors": n_neighbors,
                            "neighborhood_type": neighborhood_type,
                        }

                        # multiprocessing — równoległe powtórzenia
                        with Pool(processes=cpu_count()) as pool:
                            parallel_jobs = [
                                (solve_tsp, D, params) for _ in range(REPEATS)
                            ]
                            results_parallel = pool.map(run_single_repeat, parallel_jobs)

                        costs = [c for c, _, _ in results_parallel]
                        runtimes = [t for _, _, t in results_parallel]

                        results.append({
                            "instance": tsp_file,
                            "max_iter": max_iter,
                            "stop_no_improve": stop_no_improve,
                            "tabu_tenure": tabu_tenure,
                            "n_neighbors": n_neighbors,
                            "neighborhood_type": neighborhood_type,
                            "mean_cost": round(np.mean(costs), 3),
                            "min_cost": round(np.min(costs), 3),
                            "mean_runtime": np.mean(runtimes),
                        })


# ZAPIS WYNIKÓW
end_total = time.perf_counter()
elapsed = end_total - start_total

df = pd.DataFrame(results)
save_experiment_results(df, time_seconds=int(elapsed), subfolder="TS")


# RAPORT
print(f"\nŁączny czas eksperymentów: {elapsed/60:.2f} min ({elapsed:.2f} sek)")

print("\nPodsumowanie wyników:")
print(df.groupby("neighborhood_type")[["mean_cost", "min_cost", "mean_runtime"]]
      .mean()
      .round(3))

best = df.loc[df["mean_cost"].idxmin()]
print(f"\nNajlepsze parametry:\n{best.to_dict()}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Rozgrzewanie (kompilacja JIT jeśli dotyczy)...
Kompilacja zakończona.


Instancja: Dane_TSP_76.xlsx

Podsumowanie (pierwsze 20 wierszy):
        instance  max_iter  stop_no_improve  tabu_tenure  n_neighbors neighborhood_type  mean_cost  min_cost  mean_runtime
Dane_TSP_76.xlsx    100000            10000           15          100           two_opt 101027.051 100511.86      21.97422

Łączny czas eksperymentów: 0.47 min (28.05 sek)

Podsumowanie wyników:
                    mean_cost   min_cost  mean_runtime
neighborhood_type                                     
two_opt            101027.051  100511.86        21.974

Najlepsze parametry:
{'instance': 'Dane_TSP_76.xlsx', 'max_iter': 100000, 'stop_no_improve': 10000, 'tabu_tenure': 15, 'n_neighbors': 100, 'neighborhood_type': 'two_opt', 'mean_cost': 101027.051, 'min_cost': 100511.86, 'mean_runtime': 21.974219624998415}
